In [1]:
import requests

In [3]:
from bs4 import BeautifulSoup

In [4]:
import pandas as pd

In [5]:
from sqlalchemy import create_engine

In [10]:
headers  ={
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'en-US,en;q=0.5',
    'Connection':'keep-alive',
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; W…) Gecko/20100101 Firefox/60.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
    
}

In [12]:
xingzhengqu_cn = ['浦东','闵行','宝山','徐汇','普陀','杨浦','长宁','松江','嘉定','黄浦','静安','闸北','虹口','青浦','奉贤','金山','崇明','上海周边']

In [13]:
xingzhengqu = ['pudpng','huangpu']

In [23]:
def get_url(url,page):
    html = requests.get(url+'/pg%s'%str(page),headers = headers).text
#     获取每一条租房信息
    table = BeautifulSoup(html,'lxml').find('div',{'class':'con_box'}).find_all('li',{'data-el':'zufang'})
# 查找每个行政区的最大页码数
    table2 = BeautifulSoup(html,'lxml')
#     使用正则表达式获得最大页码数
    pattern  = re.compile('"totalPage":(.*?),".*?"')
    lastpage = re.findall(pattern,str(table2))
#     用户正则表达式获取用bs4不方便获取的信息
    pattern_mianji = re.compile("(.*?)平米.*?")
    pattern_quyu = re.compile("(.*?)租房")
    pattern_updatetime = re.compile("(.*?)\s更新")
    
    result = []
    
    for li in table:
        item = dict()
        item[u'网站'] = li.find('a').get('herf')
        item[u'描述'] = li.find('img').get('alt')
        item[u'小区'] = li.find('div', {'class': 'where'}).find('a').get_text().replace('\xa0', '')
        item[u'房型'] = li.find('div', {'class': 'where'}).find('span')[1].get_text().replace('\xa0', '')
        item[u'面积'] = re.findall(pattern_mianji, li.find('div', {'class': 'where'}).find_all('span')[3].get_text())[0]
        item[u'朝向'] = li.find('div', {'class': 'where'}).find_all('span')[4].get_text()
        item[u'地区'] = re.findall(pattern_quyu, li.find('div', {'class': 'other'}).find('a').get_text())[0]
        item[u'楼层'] = li.find('div', {'class': 'con'}).get_text().split('/')[1]
        item[u'年限'] = li.find('div', {'class': 'con'}).get_text().split('/')[2]
        item[u'价格'] = li.find('div', {'class': 'price'}).find('span', {'class': 'num'}).get_text()
        item[u'更新时间'] = re.findall(pattern_updatetime, li.find('div', {'class': 'price-pre'}).get_text())[0]
        item[u'看房人数'] = li.find('div', {'class': 'square'}).find('span', {'class': 'num'}).get_text()
        result.append(item)
        return result,last_page

In [18]:
# 定义一个函数，存储区域信息
def get_quyu_data(area,area_cn):
    url  = "https://sh.lianjia.com/zufang/%s "% area
    page =1
    last_page = get_url(url,page)[1]
    
    df = pd.DataFrame()
    while True:
        try:
            result = get_url(url,page)[0]
        except:
            print('failed')
            break
        df = df.append(pd.DataFrame(result))
        page += 1
        if page> int(''.join(last_page)):
            break
    df['区域'] = '%s' %area_cn
    return df

In [21]:
# 定义主函数
def main():
    column = ['房屋描述', '区域', '地区', '小区', '房型', '面积', '楼层', '朝向', '价格', '年限', '更新时间', '看房人数', '网站']
    frame = pd.DataFrame()
    for i in range(len(xingzhengqu)):
        quyu = get_quyu_data(xingzhengqu[i],xingzhengqu_cn[i])
        frame = frame.append(quyu)
    frame = frame.reindex(columns=column)
    frame.index = range(len(frame))
    frame.to_csv('D:/house.csv')

In [22]:
if __name__ == '__main__':
    main()

UnicodeEncodeError: 'latin-1' codec can't encode character '\u2026' in position 30: ordinal not in range(256)

In [25]:
for i  in range(36):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
